# Jiu Cai!

In [1]:
pip install yfinance --upgrade --no-cache-dir

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22410 sha256=24293feca92d4e5666895f0729b82968a2b6d3a5889b715d034dbf779ae3e796
  Stored in directory: /tmp/pip-ephem-wheel-cache-y8_1arjq/wheels/6f/ad/f4/4a269deab015672fd1ab353d6b2c3fcf64f413980737c13541
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8367 sha256=dfcd841e058abaaebd3e777888201b6175bc7e424df3695d35a16d460dccf892
  Stored in directory: /tmp/pip-ephem-wheel-cache-y8_1arjq/wheels/5e/8a/c4/59c699498647c7c94b14c87a904ca7540646107b3d94b7c320
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly==4.6.0

     |████████████████████████████████| 7.1 MB 5.8 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=00c24d13d75f02a7c342bf8e044195b042fb2d935571684148663bd6c502727f
  Stored in directory: /home/jovyan/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bs4

     |████████████████████████████████| 109 kB 5.1 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=029234c4e26030e5537244da152db0955156473da83235bee729222f97a26018
  Stored in directory: /home/jovyan/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install progressbar2

     |████████████████████████████████| 51 kB 119 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [51]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
import requests
import bs4 as bs
import progressbar
import math

def fetch_sp500_tickers():
    resp = requests.get(
        'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.content, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.rstrip())
    return tickers

def get_gain_optimistic(prices, index, max_interval, stop_percentage):
    if index + max_interval + 1 >= len(prices['Open']):
        return 0
    cost = max(prices['Open'][index], 0.001)
    sell_price = 0
    low_stop_sell = 0
    high_stop_sell = 0
    for i in range(index, index + max_interval + 1):
        if (prices['High'][i] - cost) / cost >= stop_percentage:
            return stop_percentage
        if (cost - prices['Low'][i]) / cost >= stop_percentage:
            return -stop_percentage
    return (prices['Open'][index + max_interval + 1] - cost) / cost

def get_gain_pessimistic(prices, index, max_interval, stop_percentage):
    if index + max_interval + 1 >= len(prices['Open']):
        return 0
    cost = max(prices['Open'][index], 0.001)
    sell_price = 0
    low_stop_sell = 0
    high_stop_sell = 0
    for i in range(index, index + max_interval + 1):
        if (cost - prices['Low'][i]) / cost >= stop_percentage:
            return -stop_percentage
        if (prices['High'][i] - cost) / cost >= stop_percentage:
            return stop_percentage
    return (prices['Open'][index + max_interval + 1] - cost) / cost

def get_trend(prices, start_index, end_index):
    ascending = True
    descending = True
    for i in range(start_index, end_index + 1):
        if prices['Open'][i] < prices['Close'][i]:
            descending = False
        elif prices['Open'][i] > prices['Close'][i]:
            ascending = False
    if ascending:
        return 1
    if descending:
        return -1
    return 0

def is_shooting_star(prices, index, threshold):
    return ((index > 4) and 
            (prices['Open'][index] <= prices['Close'][index]) and 
            ((prices['High'][index] - prices['Close'][index]) / max(prices['Close'][index] - prices['Open'][index], 0.001) >= threshold) and 
            ((prices['High'][index] - prices['Close'][index]) / max(prices['Open'][index] - prices['Low'][index], 0.001) >= threshold) and 
            (get_trend(prices, index - 4, index - 1) == 1))

In [ ]:
tickers = fetch_sp500_tickers()
shooting_star_ticker_list = {}
print('Start loading tickers...')

with progressbar.ProgressBar(max_value=len(tickers)) as bar:
    i = 0
    for ticker in tickers:
        i += 1
        bar.update(i)
        shooting_stars = []
        price = yf.Ticker(ticker)
        period = '60d' # m = min, h = hour, d = day, wk = week, mo=month
        interval = '15m' # valid intervals 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
        prices = price.history(period=period, interval=interval)
        for index in range(prices.shape[0]):
            if is_shooting_star(prices, index, 2) == True:
                # Since we are shorting stock, we need to be "optimistic" for the worst case.
                shooting_stars.append((index, get_gain_optimistic(prices, index + 1, 4, 0.05)))
        if shooting_stars:
            shooting_star_ticker_list[ticker] = shooting_stars
shooting_star_ticker_list
            

N/A% (0 of 505) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Start loading tickers...


  3% (18 of 505) |                       | Elapsed Time: 0:00:08 ETA:   0:04:01

In [49]:
total_gain = 0
positive = 0
negative = 0
for _, shooting_star_tickers in shooting_star_ticker_list.items():
    for pair in shooting_star_tickers:
        gain = pair[1]
        if math.isnan(gain):
            gain = 0
        total_gain += gain
        if gain > 0:
            positive += 1
        else:
            negative += 1
total_gain

print("Total gain: %s; Positive number: %d; Negative number: %d" % (total_gain, positive, negative))

Total gain: 1.6623976888383334; Positive number: 63; Negative number: 41


In [50]:
price = yf.Ticker('REGN')
period = '360d' # m = min, h = hour, d = day, wk = week, mo=month
interval = '5d' # valid intervals 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
prices = price.history(period=period, interval=interval)

plot_list = np.array([47])
plot_list = np.concatenate((plot_list, plot_list-1, plot_list-2, plot_list-3,plot_list+1, plot_list+2, plot_list+3,plot_list+4, plot_list+5))
plot_list = np.sort(plot_list)

fig = go.Figure(data=[go.Candlestick(x=prices.index[plot_list],
                open=prices['Open'][plot_list], high=prices['High'][plot_list],
                low=prices['Low'][plot_list], close=prices['Close'][plot_list])
                     ])
fig.show()
